# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

##### Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
-	сalls — количество звонков,
-	minutes — суммарная длительность звонков в минутах,
-	messages — количество sms-сообщений,
-	mb_used — израсходованный интернет-трафик в Мб,
-	is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 1. Откроем и изучим файл

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, mean_squared_error, precision_recall_fscore_support
from sklearn.dummy import DummyClassifier

In [2]:
# users = pd.read_csv('/Users/Mikalai/Documents/Data Science/Project Sprint 9/users_behavior.csv')
users = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
display(users.head(10)) # структуру таблицы
display(users.describe()) # разброс значений, среднее и медиана 
display(users.info()) # состав таблицы, формат столбцов, наличие пропусков 

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

В таблице данных 3214 записей. Пропусков нет.
В столбцах calls, messages и is_ultra содержится целочисленная информация. Заменим тип данных в них на int для уменьшения занимаемого места.

In [4]:
users['calls'] = users['calls'].astype('int')
users['messages'] = users['messages'].astype('int')
users['is_ultra'] = users['is_ultra'].astype('int')
display(users.head())
display(users.info())

,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int32  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int32  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int32  
dtypes: float64(2), int32(3)
memory usage: 88.0 KB


None

### Выводы по изучению данных

Данные не нуждаются в предобработке.

Пропуски отсутствуют.

Сменили тип данных в столбцах calls, messages is_ultra с float на int.

## 2. Разделим данные на выборки

Разобьём данные на 2 выборки с учётом распределения классов:
- обучающаяся выборка, которая равна 75% от общих данных
- тестовая выборка, которая равна 25% от общих данных

In [5]:
users_train, users_test = train_test_split(users, test_size=0.25, random_state=0, stratify=users.is_ultra)

Проверим разбивку данных

In [6]:
print(f'Размер обучающей выборки составляет: {users_train.shape[0] / len(users):.1%}')
print(f'Размер тестовой выборки составляет: {users_test.shape[0] / len(users):.1%}')

Размер обучающей выборки составляет: 75.0%
Размер тестовой выборки составляет: 25.0%


### Выводы по разделению данных

Разбили данные на соотношение 3:1 (3 - обучающая выборка, 1 - тестовая выборка)

## 3. Исследуем качество разных моделей

Перед исследованием моделей машинного обучения создадим переменные для признаков и целевого признака для каждой выборки

In [7]:
features_train = users_train.drop('is_ultra', axis=1)
target_train = users_train['is_ultra']
features_test = users_test.drop('is_ultra', axis=1)
target_test = users_test['is_ultra']

На основании данных и стоящей перед нами задачи классификации построим следущие модели:
- дерево решений
- случайный лес
- логистическая регрессия

После выберем лучшую из них с максимально большим значением accuracy

### 3.1 Исследуем модель дерево решений

In [8]:
clf_dtc = DecisionTreeClassifier()
parametrs_dtc = {'max_depth': range (1,11)}
grid_dtc = GridSearchCV(clf_dtc, parametrs_dtc, cv=5, n_jobs=-1)
grid_dtc.fit(features_train, target_train)
print('Точность лучшей модели', grid_dtc.best_score_, 'с параметрами', grid_dtc.best_params_)

Точность лучшей модели 0.7950207468879669 с параметрами {'max_depth': 3}


### 3.2 Исследуем модель случайный лес

In [9]:
clf_rfc = RandomForestClassifier()
parametrs_rfc = {'n_estimators': range (10, 41, 10), 'max_depth': range (1,13)}
grid_rfc = GridSearchCV(clf_rfc, parametrs_rfc, cv=5, n_jobs=-1)
grid_rfc.fit(features_train, target_train)
print('Точность лучшей модели', grid_rfc.best_score_, 'с параметрами', grid_rfc.best_params_)

Точность лучшей модели 0.8045643153526971 с параметрами {'max_depth': 8, 'n_estimators': 10}


### 3.3 Исследуем модель логистическая регрессия

In [10]:
clf = LogisticRegressionCV(Cs=10, cv=5, random_state=0).fit(features_train, target_train)
clf.scores_[1].mean(axis=0).max()

0.7406639004149378

### Выводы по исследованию моделей

Построили 3 модели (дерево решений, случайный лес, логистическая регрессия), которые показали:
- У модели "дерево решений" при глубине в 3, точность наилучшей модели на валидационной выборке равна: 0.79
- У модели "случайноый лес" при 10 деревьях и глубине 8, точность наилучшей модели на валидационной выборке равна: 0.80
- У модели "логистическая регрессия" точность модели на валидационной выборке равна: 0.74

Исходя из этих данных можно сделать вывод, что лучшей моделью с максимально большим значением accuracy является модель случайный лес при 10 деревьях с глубиной 8.

## 4. Проверим качество модели на тестовой выборке

In [11]:
best_model = grid_rfc.best_estimator_
prediction = best_model.predict(features_test)
accuracy = accuracy_score(prediction, target_test)
print('Точность тестовой модели равна:', accuracy)

Точность тестовой модели равна: 0.8072139303482587


## Вывод по исследованию модели на тестовой выборке

Таким образом, точность тестовой модели составила 0.81, что больше 0.75 по условию задания

## 5. Проверим модель на адекватность

Чтобы оценить нашу модель на адекватность, посмотрим не предсказывает ли модель одни 0 или 1. Для этого посмотрим как распределены пользователи по тарифным планам

In [12]:
users.is_ultra.value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Таким образом, если бы наша модель предсказывала только 1 или 0, то результат её точности был бы равен либо 0.694, либо 0.306. Следовательно, наша модель предсказывает верно.

### Общий вывод 

Лучшая модель для задачи классификации по выбору подходящего тарифа - модель "случайный лес". Данная модель показала точность тестовой модели 0.81, а так же прошла проверку на адекватность.